## *Tarea M53 - Bryan Alberto Coronado García*
---

### 1) Conexión desde Python al servicio Spark donde se tiene la información.
### 2) Importación de datos desde Spark.

In [3]:
from pyspark.sql import SparkSession
import sys
import warnings
warnings.filterwarnings('ignore')

spark = SparkSession.builder.appName('TareaM53').getOrCreate()

df = spark.read.option('header', True).csv('/Users/bryancoronado/Desktop/EBAC/kc_house_data.csv')
df.show(10)

+----------+---------------+----------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|        id|           date|     price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
+----------+---------------+----------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|7129300520|20141013T000000|    221900|       3|        1|       1180|    5650|     1|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|
|6414100192|20141209T000000|    538000|       3|     2.25|       2570|    7242|     2|         0|   0|        3|    7|      2170|       

In [5]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sqft_above: string (nullable = true)
 |-- sqft_basement: string (nullable = true)
 |-- yr_built: string (nullable = true)
 |-- yr_renovated: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- sqft_living15: string (nullable = true)
 |-- sqft_lot15: string (nullable = true)



### 3) Ejecución de comandos Python que traigan la misma información obtenida en los videos.

#### partitionBy()

In [ ]:
# Generación de particiones por ZipCode
df.write.option('header', True).partitionBy('zipcode').mode('overwrite').csv('/Users/bryancoronado/Desktop/EBAC/Partitions/kc_house_data_zipcodes')

In [8]:
# Generación de particiones por ZipCodes y Waterfront
df.write.option('header', True).partitionBy('zipcode', 'waterfront').mode('overwrite') \
    .csv('/Users/bryancoronado/Desktop/EBAC/Partitions/ZipCodes_Waterfront')

In [9]:
# Generación de particiones controlando la cantidad por partición
df.write.option('header', True).partitionBy('zipcode').mode('overwrite') \
    .option('maxRecordsPerFile', 30) \
    .csv('/Users/bryancoronado/Desktop/EBAC/Partitions/zipcode_control')

#### Coalesce & Repartition

In [ ]:
# Se genera nueva sesion para evitar conflictos con la anterior
spark = SparkSession.builder.appName('CoalRepart').getOrCreate()

# Se procesa un nuevo DataFrame para aplicar coalesce y repartition
df2 = spark.read.option('header', True).csv('/Users/bryancoronado/Desktop/EBAC/kc_house_data.csv')

# Muestra del DataFrame
df2.show(10)

+----------+---------------+----------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|        id|           date|     price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
+----------+---------------+----------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|7129300520|20141013T000000|    221900|       3|        1|       1180|    5650|     1|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|
|6414100192|20141209T000000|    538000|       3|     2.25|       2570|    7242|     2|         0|   0|        3|    7|      2170|       

In [12]:
# Ejemplos de Repartition

df2.repartition(10).write.mode('overwrite').option('header', True).csv('/Users/bryancoronado/Desktop/EBAC/Partitions/repartitioned_data')

In [13]:
# Ejemplos de Coalesce

df2.coalesce(5).write.mode('overwrite').option('header', True).csv('/Users/bryancoronado/Desktop/EBAC/Partitions/coalesced_data')

#### PyArrow

In [17]:
# Libreria PyArrow para leer el archivo CSV
from pyarrow import csv
import pyarrow as pa
import pandas as pd 

#Lectura del archivo CSV usando PyArrow
path_archivo = '/Users/bryancoronado/Desktop/EBAC/kc_house_data.csv'
tabla = csv.read_csv(path_archivo)

# Visualización de la tabla
tabla

pyarrow.Table
id: int64
date: string
price: double
bedrooms: int64
bathrooms: double
sqft_living: int64
sqft_lot: int64
floors: double
waterfront: int64
view: int64
condition: int64
grade: int64
sqft_above: int64
sqft_basement: int64
yr_built: int64
yr_renovated: int64
zipcode: int64
lat: double
long: double
sqft_living15: int64
sqft_lot15: int64
----
id: [[7129300520,6414100192,5631500400,2487200875,1954400510,...,2125049131,5318101765,6624300110,5160700035,1472800220],[3755000090,259601010,7939000090,2675600025,5700000975,...,1931300110,8624700015,6149700315,2064800880,3902300450],[1332300110,7663700610,7520000330,1521049156,6648760150,...,263000018,6600060120,1523300141,291310100,1523300157]]
date: [["20141013T000000","20141209T000000","20150225T000000","20141209T000000","20150218T000000",...,"20140729T000000","20140602T000000","20140623T000000","20150422T000000","20140909T000000"],["20141006T000000","20150414T000000","20140929T000000","20150327T000000","20140819T000000",...,"201407

In [19]:
# Longitud y tipo de la tabla
print(len(tabla))
print(type(tabla))

21613
<class 'pyarrow.lib.Table'>


In [ ]:
# Transformación de la tabla a DataFrame de Pandas
df_pandas = tabla.to_pandas()
df_pandas.sample(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
14050,7853220210,20140922T000000,563500.0,4,2.50,2780,7838,2.0,0,3,...,9,2780,0,2004,0,98065,47.5312,-121.861,3160,7848
220,5611500140,20140822T000000,686000.0,4,2.50,2760,6440,2.0,0,0,...,10,2760,0,1999,0,98075,47.5836,-122.026,3070,8127
18035,8856970530,20141208T000000,326995.0,3,2.50,1860,5321,2.0,0,0,...,7,1860,0,2000,0,98038,47.3848,-122.033,1940,5205
9426,5149800040,20140918T000000,255000.0,4,2.00,2560,12155,1.0,0,0,...,7,1350,1210,1960,0,98003,47.3326,-122.323,1790,11906
12259,1786640030,20150202T000000,359950.0,3,2.00,1790,7212,1.0,0,0,...,8,1790,0,1998,0,98042,47.3898,-122.154,2330,7212
20017,1025039168,20140923T000000,290000.0,1,0.75,740,1284,1.0,0,0,...,6,740,0,1928,0,98107,47.6741,-122.406,1430,3988
11257,3761100045,20140618T000000,3000000.0,4,4.25,4850,12445,2.0,1,4,...,10,3850,1000,1989,0,98034,47.7011,-122.244,3350,12210
6751,993000950,20140906T000000,320000.0,2,1.50,1110,1200,3.0,0,0,...,8,1110,0,2000,0,98103,47.6931,-122.342,1110,1363
18014,2064800880,20150201T000000,301500.0,3,1.00,1410,7419,1.0,0,0,...,7,1050,360,1969,0,98056,47.5340,-122.173,1800,8000
21480,567000775,20140912T000000,449000.0,2,2.50,1460,1296,2.0,0,0,...,8,1160,300,2008,0,98144,47.5923,-122.296,1460,1296


In [22]:
# Agrupación de una tabla y transformación a DataFrame de Pandas
pd.set_option('display.float_format', lambda x: '%.2f' % x)
tabla_agrupada = tabla.group_by('zipcode').aggregate([('price', 'mean')])
df_agrupado_pandas = tabla_agrupada.to_pandas()
df_agrupado_pandas.sample(10)

,zipcode,price_mean
57,98109,879623.62
18,98040,1194230.02
26,98058,353608.64
41,98034,521652.86
10,98007,617105.09
43,98116,618634.17
39,98004,1355927.08
1,98125,469455.77
63,98031,300539.89
13,98126,424706.36


In [28]:
# Agregación de una columna y transformación a DataFrame de Pandas
tab_agg = tabla.append_column('NewCol', pa.array(['0'] * len(tabla), pa.string()))
df_agg = tab_agg.to_pandas()
df_agg.sample(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,NewCol
6842,2426059071,20141021T000000,675000.00,3,2.50,2320,98445,1.00,0,0,...,1380,940,1978,0,98072,47.73,-122.12,2830,54014,0
10337,3579000800,20141024T000000,509000.00,4,2.50,2600,9355,1.00,0,0,...,1840,760,1983,0,98028,47.74,-122.25,2250,7691,0
20977,7889601165,20140826T000000,268000.00,3,2.50,1700,2250,2.00,0,0,...,1700,0,2014,0,98168,47.49,-122.33,1520,4500,0
18345,2722049246,20141114T000000,280000.00,3,2.00,1640,13249,1.00,0,0,...,1640,0,1995,0,98032,47.36,-122.28,1640,9240,0
19351,2525049113,20140725T000000,1950000.00,4,3.50,4065,18713,2.00,0,0,...,4065,0,1987,0,98039,47.62,-122.24,3070,18713,0
12629,1219000120,20150402T000000,340000.00,4,1.00,1140,13440,1.00,0,0,...,1140,0,1944,0,98166,47.46,-122.34,1450,7560,0
15824,1061500630,20150205T000000,359900.00,5,2.75,2790,7600,1.50,0,0,...,2790,0,1965,0,98056,47.50,-122.17,1480,7600,0
9600,8122600165,20141015T000000,273000.00,3,1.00,1500,6250,1.00,0,0,...,890,610,1945,0,98126,47.54,-122.37,1210,6250,0
6478,126039323,20150224T000000,417500.00,5,1.75,2060,10911,1.00,0,0,...,1360,700,1954,0,98177,47.78,-122.36,2060,9688,0
15484,4029400080,20150406T000000,383000.00,4,2.50,1850,8310,1.00,0,0,...,1200,650,1962,0,98155,47.77,-122.29,1840,10080,0


#### Parquet

In [30]:
import pyarrow.parquet as pq

df = pd.DataFrame({'1linea': [1, 2, 3], '2linea': ['palabra1', 'palabra2', 'palabra3'], '3linea': [1.1, 2.2, 3.3]}, index = list('abc'))
df

,1linea,2linea,3linea
a,1,palabra1,1.10
b,2,palabra2,2.20
c,3,palabra3,3.30


In [31]:
parquet_file = pa.Table.from_pandas(df)
pq.write_table(parquet_file, '/Users/bryancoronado/Desktop/EBAC/Partitions/parquet_example.parquet')